In [46]:
import pandas as pd

In [47]:
df = pd.read_csv('../csv/BND.csv', index_col = 'Date')

df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-04-10,75.070000,75.260002,75.000000,75.239998,51.523693,35000
2007-04-11,75.160004,75.290001,75.029999,75.040001,51.386742,87700
2007-04-12,75.059998,75.080002,74.959999,75.029999,51.379879,78100
2007-04-13,75.040001,75.070000,74.849998,74.910004,51.297688,18000
2007-04-16,74.989998,74.989998,74.940002,74.980003,51.345627,52700


In [48]:
price_df = df.loc[:, ['Adj Close']]

price_df.head(1)

,Adj Close
Date,
2007-04-10,51.523693


In [49]:
# 볼린저밴드: 상단, 하단 밴드를 설정 후 주가가 하단 밴드 밑으로 내려가면 매수 / 주가가 상단 밴드 위로 올라가면 매도
# 이동평균선 파생변수 생성 → 데이터양 20개의 평균을 구해서 컬럼에 대입

price_df['center'] = price_df['Adj Close'].rolling(20).mean()

price_df.iloc[18 : 25]

,Adj Close,center
Date,,
2007-05-04,51.679100,NaN
2007-05-07,51.713379,51.556713
2007-05-08,51.720268,51.566542
2007-05-09,51.658497,51.580130
2007-05-10,51.706562,51.596464
2007-05-11,51.637943,51.613477
2007-05-14,51.603649,51.626378


In [50]:
# 상단 밴드 파생변수 생성(이동평균선 + 2 * 표준편차)

price_df['ub'] = price_df['center'] + 2 * price_df['Adj Close'].rolling(20).std()
price_df.iloc[18 : 25]

,Adj Close,center,ub
Date,,,
2007-05-04,51.679100,NaN,NaN
2007-05-07,51.713379,51.556713,51.807910
2007-05-08,51.720268,51.566542,51.827492
2007-05-09,51.658497,51.580130,51.829713
2007-05-10,51.706562,51.596464,51.833301
2007-05-11,51.637943,51.613477,51.804375
2007-05-14,51.603649,51.626378,51.770106


In [51]:
# 하단 밴드 파생변수 생성(이동평균선 - 2 * 표준편차)

price_df['lb'] = price_df['center'] - 2 * price_df['Adj Close'].rolling(20).std()
price_df.iloc[18 : 25]

,Adj Close,center,ub,lb
Date,,,,
2007-05-04,51.679100,NaN,NaN,NaN
2007-05-07,51.713379,51.556713,51.807910,51.305517
2007-05-08,51.720268,51.566542,51.827492,51.305592
2007-05-09,51.658497,51.580130,51.829713,51.330546
2007-05-10,51.706562,51.596464,51.833301,51.359627
2007-05-11,51.637943,51.613477,51.804375,51.422579
2007-05-14,51.603649,51.626378,51.770106,51.482650


In [52]:
# 시작 시간

start_time = '2010-01-02'
price_df = price_df.loc[start_time:]

price_df

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,60.611969,60.826958,61.336124,60.317793
2010-01-05,60.789135,60.819153,61.325461,60.312844
2010-01-06,60.766037,60.802128,61.290324,60.313932
2010-01-07,60.719822,60.777802,61.228214,60.327389
2010-01-08,60.781410,60.759625,61.175685,60.343565
...,...,...,...,...
2019-06-18,82.397118,81.737070,82.743959,80.730181
2019-06-19,82.676468,81.836350,82.799510,80.873191
2019-06-20,82.806168,81.934650,82.870640,80.998661


In [53]:
# 'trade' 파생변수 생성, 값들은 비어있는 값

price_df['trade'] = ''

price_df

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,60.611969,60.826958,61.336124,60.317793,
2010-01-05,60.789135,60.819153,61.325461,60.312844,
2010-01-06,60.766037,60.802128,61.290324,60.313932,
2010-01-07,60.719822,60.777802,61.228214,60.327389,
2010-01-08,60.781410,60.759625,61.175685,60.343565,
...,...,...,...,...,...
2019-06-18,82.397118,81.737070,82.743959,80.730181,
2019-06-19,82.676468,81.836350,82.799510,80.873191,
2019-06-20,82.806168,81.934650,82.870640,80.998661,


In [54]:
# 구매 내역 데이터를 삽입
# 조건: 상단 밴드보다 종가가 높은 경우
#       if 구매 상태
#           매도 trade = ''
#       else
#           아무 행동 X trade = ''
#
#      하단 밴드보다 종가가 낮은 경우
#       if 구매 상태
#           구매 상태 유지 trade = 'buy'
#       else
#           구매 trade = 'buy'
#
#      상단 밴드와 하단 밴드 사이에 종가가 존재하면
#       if 구매 상태
#           현 상태 유지 trade = 'buy'
#       else
#           현 상태 유지 trade = ''

In [55]:
for i in price_df.index :
    if price_df.loc[i, 'Adj Close'] > price_df.loc[i, 'ub'] :
        price_df.loc[i, 'trade'] = ''
    
    elif price_df.loc[i, 'lb'] > price_df.loc[i, 'Adj Close'] :
        price_df.loc[i, 'trade'] = 'buy'

    elif price_df.loc[i, 'ub'] >= price_df.loc[i, 'Adj Close'] and price_df.loc[i, 'Adj Close'] >= price_df.loc[i, 'lb'] :
        if price_df.shift(1).loc[i, 'trade'] == 'buy' :         # shift(1) = 전 행의 데이터
            price_df.loc[i, 'trade'] = 'buy'

        else :
            price_df.loc[i, 'trade'] = ''

price_df.value_counts('trade')      # buy 값이 제대로 들어갔는지 확인

trade
       1568
buy     816
dtype: int64

In [56]:
# 손익 계산

rtn = 1.0
buy = 0.0
sell = 0.0

price_df['return'] = 1

for i in price_df.index :
    if price_df.loc[i, 'trade'] == 'buy' and price_df.shift(1).loc[i, 'trade'] == '' :      # 구매하는 경우
        buy = price_df.loc[i, 'Adj Close']

        print('진입일: ', i, '진입 가격: ', buy)

    elif price_df.loc[i, 'trade'] == '' and price_df.shift(1).loc[i, 'trade'] == 'buy' :    # 매도하는 경우
        sell = price_df.loc[i, 'Adj Close']
        rtn = (sell - buy) / buy + 1        # 손익 계산식
        price_df.loc[i, 'return'] = rtn

        print('청산일: ', i, '진입 가격: ', buy, '청산 가격: ', sell, '| return: ', round(rtn, 4))

    if price_df.loc[i, 'trade'] == '' :         # buy, sell 변수 초기화
        buy = 0.0
        sell = 0.0


acc_rtn = 1.0

for i in price_df.index :
    rtn = price_df.loc[i, 'return']
    acc_rtn = acc_rtn * rtn         # 누적곱
    price_df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익률: ', round(acc_rtn, 4))

진입일:  2010-03-25 진입 가격:  61.205368
청산일:  2010-05-04 진입 가격:  61.205368 청산 가격:  62.186432 | return:  1.016
진입일:  2010-09-10 진입 가격:  64.517815
청산일:  2010-09-28 진입 가격:  64.517815 청산 가격:  65.29863 | return:  1.0121
진입일:  2010-11-12 진입 가격:  64.902885
청산일:  2011-03-16 진입 가격:  64.902885 청산 가격:  65.105652 | return:  1.0031
진입일:  2011-06-29 진입 가격:  66.049957
청산일:  2011-08-01 진입 가격:  66.049957 청산 가격:  67.197464 | return:  1.0174
진입일:  2011-11-30 진입 가격:  68.584297
청산일:  2011-12-13 진입 가격:  68.584297 청산 가격:  69.032143 | return:  1.0065
진입일:  2012-03-14 진입 가격:  69.140411
청산일:  2012-04-10 진입 가격:  69.140411 청산 가격:  70.015167 | return:  1.0127
진입일:  2012-08-15 진입 가격:  71.121567
청산일:  2013-02-25 진입 가격:  71.121567 청산 가격:  71.270378 | return:  1.0021
진입일:  2013-05-10 진입 가격:  71.452927
청산일:  2013-09-18 진입 가격:  71.452927 청산 가격:  69.627502 | return:  0.9745
진입일:  2013-12-24 진입 가격:  69.648918
청산일:  2014-02-03 진입 가격:  69.648918 청산 가격:  70.937912 | return:  1.0185
진입일:  2014-09-10 진입 가격:  72.17923
청산일:  2014-10-

In [57]:
# 볼린저밴드 모듈 생성
# 1. bollinger 폴더 생성
# 2. '__init__' 파일 생성
# 3. 클래스 생성
# 4. 데이터프레임, 기준이 되는 컬럼을 '__init__' 함수에서 인자로 받아오는 함수
# 5. 이동평균선, 상단 밴드, 하단 밴드, trade == '' 생성하는 함수 생성
# 6. 시작시간은 인자값으로 받아와서 데이터의 개수 필터링
# 7. 구매 상태를 삽입하는 함수 생성
# 8. 손익 계산하는 함수 생성